# KET VR5 SAC
### Jonathan Ramos 8/20/2024
If I am understanding the experimental timeline correctly, these animals recieved 2 weeks of training on an FR1 schedule, then recieved an IP injection of ketamine prior to VR5 reactivation. Then 40(?) minutes after reactivation were sacked. In this ipynb I will load in the data which is distributed across ~200 csvs, perform some data wrangling and ultimately ensure complete subraph colocalization. 

The idea here is to treat each image as a graph of nodes. Each cell of each stain type is considered as a node and nodes (of different stain types) are connected via vertices if they are colocalized, and we may then consider that a given cell is expressing 2 or more of the targets identifed by the IHC. This means that colocalization produces subgraphs that are both complete (fully connected; if a PV cell says it's colocalized with a cFos cell, then that cFos cell must also say it's colocalized with that PV cell) and disjoint (if a PV_1 says it's colocalized with cFos_1, then PV_2 cannot also be colocalized with cFos_1). 

These complete subgrpahs are also called "cliques" in graph theory. There are several algorithms to identify complete subgraphs in a graph network however, here I've just implemented by hand a quick and dirty breadth first search. Next it is likely the case that there will be some mismatched cells that violate the outlined requirements (subgraphs must be complete and disjoint) and so I will "break ties" by computing distances and assigning colocalization to the groupings that are most closely localized with each other. In this data set I found only 3way and 4way ties, but for completeness I have also included the code to break 5way ties (previously found in the other larger ketamine dataset).

Overall this data was successfully cleaned yielding the expected numbers of double and triple labeled cells shown below. For example, if we have 100 PV on WFA cells, then we therefore also expected exactly 100 WFA on PV cells. Since triple labeled cells were the highest order of colocalization in this set (here we only stained for PV, WFA and cFos), we expect all combinations of triple labeled cells to have exacty the same count.


\
double labeled ns:\
PV on WFA:    477\
WFA on PV:    477

PV on cFos:    292\
cFos on PV:    292

WFA on cFos:    345\
cFos on WFA:    345


triple PV,WFA,cFos ns:\
PV  : 150\
WFA  : 150\
cFos  : 150

one last thing to add: this code could be refactored as a module for use in scripts, but since data wrangling is often a very specific process with conventions that vary on a project by project basis (which is not required to be scalable in our case), I prefer to perform these tasks in a notebook environment for readability, reproducibility, sharing, as well as quick and easy visualization. 

In [1]:
import os
import shutil
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## load in data, some data cleaning

In [2]:
files = sorted([f for f in os.listdir('data/') if '.csv' in f])
df_data = [pd.read_csv(f'data/{f}') for f in files]

doubles = [df for df in df_data if len([col for col in df.columns if 'coloc' in col]) == 2]
triples = [df for df in df_data if len([col for col in df.columns if 'coloc' in col]) == 3]

def preprocessing(coloc_dfs):
    '''
    small fn to automate some of the preprocessing steps that must be done
    on each df before we can concat: removing unncessary whitespaces,
    dropping rows without an intensity measurement

    note that this takes in a list of dataframes and
    returns a new list of modified dataframes
    '''
    # remove leading whitespace from col names
    for df in coloc_dfs:
        df.columns = [col.replace(' ', '') for col in df.columns]

    # drop rows without intensity measurement
    coloc_dfs = [df[~df['mean-background'].isna()] for df in coloc_dfs]

    # remove leading whitespace from filenames (note access via .loc to avoid setting with copy)
    for df in coloc_dfs:
        df.loc[:, 'filename'] = df.filename.str.replace(' ', '')

    return coloc_dfs

###### should I use the mean-backgorund col or make my own? #######
doubles = preprocessing(doubles)
triples = preprocessing(triples)

### renaming stain type labels

In [3]:
def infer_stains(df):
    '''
    standardizes strings in "stain" col based on filename. Naming schemes here 
    will vary on a project by project basis, please check that the inference 
    applied by _rule() helper fn is consistent.
    '''
    def _rule(fname):
        '''
        small helper fn which returns target stain string from input filename;
        to pass through col apply()
        '''
        if 'PV' in fname:
            return 'PV'
        elif 'WFA' in fname:
            return 'WFA'
        elif 'cfos' in fname:
            return 'cFos'
        else:
            return 'check your stains'

    df['stain'] = df.filename.apply(_rule)
    assert 'check your stains' not in df.stain

    return df

doubles = [infer_stains(df) for df in doubles]
triples = [infer_stains(df) for df in triples]

### renaming coloc col names

In [4]:
def rename_coloc(df):
    '''
    checks that the ordering of data by stain type is consistent, renames
    coloc cols accordingly. Here we assume that PV always comes first, (and
    (is therefore also the first coloc_col), then WFA, then cFos. If this
    assumption is not met, one of the assetions will fail.
    '''
    stains = df.stain.unique()
    col, ind = np.unique(stains, return_index=True)
    coloc_cols = [f'coloc_w_{stains[i]}' for i in sorted(ind)]
    d_ind = dict(zip(col, ind))

    # check that our ordering is correct
    try:
        assert d_ind['PV'] < d_ind['WFA']
    except:
        try:
            assert d_ind['WFA'] < d_ind['cFos']
        except:
            try:
                assert d_ind['PV'] < d_ind['cFos']
            except:
                print('Check your ordering assumption')
                print(d_ind)

    # replace colnames 
    for i, col in enumerate(coloc_cols):
        df = df.rename(columns={df.columns[i+1]: coloc_cols[i]})

    return df

doubles = [rename_coloc(df) for df in doubles]
triples = [rename_coloc(df) for df in triples]

### relabeling roi_id strings

In [5]:
def update_roi_ids(df):
    # as of 8/19/2024, the followign line now produces a future deprecation warning;
    # issue is related to changing dtypes in replace() call. we don't care about 
    # this warning (downcasting behavior is intended)
    df = df.replace('-', np.nan)

    # join roi_id and stain cols, more informative roi_id strings 
    # (this will be helpful later since roi_ids start from 0 for each image)
    df['roi_id'] = df.apply(lambda x: '_'.join([x.roi_id, x.stain]), axis=1)

    # add stain type label to end of coloc roi_ids if coloc stain type label is not null
    coloc_cols = [col for col in df.columns if 'coloc' in col]
    for col in coloc_cols:
        df[col] = df[col].apply(lambda x: '_'.join([x, col.split('_')[-1]]) if not pd.isnull(x) else x)

    return df

doubles = [update_roi_ids(df) for df in doubles]
triples = [update_roi_ids(df) for df in triples]

/var/folders/b2/3h2lpxx14kgb12pp_7pltxnc0000gn/T/ipykernel_29508/2466839461.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('-', np.nan)


## aggregate across images, build adjacency list

In [6]:
def get_adjacency(df, coloc_type):
    '''
    If we consider each cell as a node in directed graph, two nodes were only considered 
    as adjacent (colocalized) if they form a complete subgraph (i.e. a cFos cell says its
    paired with an PV cell, and that PV cell also says its paired with that cFos cell)
    '''
    sort_order = {'PV':0, 'WFA':1, 'cFos':2}
    
    if not coloc_type in set(['double', 'triple']):
        raise ValueError('coloc_type must be either "double" or "triple')

    if coloc_type == 'double':
        df['grouping'] = df.apply(\
            lambda x: tuple(sorted([y for y in (x.iloc[[1,2,3]]) if not pd.isnull(y)],\
                key=lambda z: sort_order[z.split('_')[-1]])), axis=1)

    if coloc_type == 'triple':
        df['grouping'] = df.apply(\
            lambda x: tuple(sorted([y for y in (x.iloc[[1,2,3,4]]) if not pd.isnull(y)],\
                key=lambda z: sort_order[z.split('_')[-1]])), axis=1)

    return df.drop(columns=[col for col in df.columns if 'coloc' in col], axis=1)

df_double_adj = pd.concat([get_adjacency(df, coloc_type='double') for df in doubles])
df_triple_adj = pd.concat([get_adjacency(df, coloc_type='triple') for df in triples])

In [7]:
### concat all colocalization data across double and triple sets
# duplicates will be removed in the groupby/aggregation by image_name and rat_n
print(len(df_double_adj))
print(len(df_triple_adj))
df_adj = pd.concat([df_double_adj, df_triple_adj])

21504
10910


### add image name, rat_n cols
To proceed with enforcing complete subgraph colocalization, I need at least rat_n and image_name cols. I don't have the cohort key at the moment but will add that data to the set later when I receive it.

In [8]:
df_adj['rat_n'] = df_adj.filename.apply(lambda x: '_'.join(x.split('_')[:2]))
df_adj['image_name'] = df_adj.filename.apply(lambda x: '_'.join(x.split('_')[:4]))

###### GET COHORT KEY
# df_adj['treatment'] = 
# df_adj['react'] =

# set types (i could have done this earlier but these were read in as str)
df_adj['CoM_x'] = df_adj.CoM_x.astype(float)
df_adj['CoM_y'] = df_adj.CoM_y.astype(float)
df_adj['background'] = df_adj.background.astype(float)
df_adj['mean_intensity'] = df_adj.mean_intensity.astype(float)

In [9]:
sort_order = {'PV':0, 'WFA':1, 'cFos':2}
agg_groupings = df_adj.groupby(['image_name', 'roi_id']).grouping.sum()\
    .apply(lambda x: tuple(sorted(sorted(list(set(x))), key=lambda y: sort_order[y.split('_')[-1]])))\
    .reset_index().reset_index().drop('index', axis=1)\
    .rename(columns={'grouping':'agg_grouping'})

df_agg_groupings = agg_groupings\
    .merge(df_adj.drop_duplicates(subset=['image_name', 'roi_id']), on=['image_name', 'roi_id'], how='left')

df_agg_groupings

,image_name,roi_id,agg_grouping,stain,CoM_x,CoM_y,pixel_area,background,mean_intensity,median_intensity,...,circularity,aspect_ratio,roundness,solidity,skewness,kurtosis,filename,analysis_date,grouping,rat_n
0,KET10_1_r1c1_left,0-000-00000_PV,"(0-000-00000_PV,)",PV,502.88,236.10,80.0,88.7182,127.6000,126.0,...,1.0000,1.0000,1.0000,0.9524,0.6552,1.0906,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00000_PV,)",KET10_1
1,KET10_1_r1c1_left,0-000-00001_PV,"(0-000-00001_PV,)",PV,491.89,306.14,80.0,88.7182,114.3500,115.0,...,0.9985,1.5163,0.6595,1.0526,0.0226,0.1296,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00001_PV,)",KET10_1
2,KET10_1_r1c1_left,0-000-00002_PV,"(0-000-00002_PV,)",PV,436.16,167.66,80.0,88.7182,129.1875,136.0,...,1.0000,1.0000,1.0000,0.9524,-0.4449,-0.7357,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00002_PV,)",KET10_1
3,KET10_1_r1c1_left,0-000-00003_PV,"(0-000-00003_PV,)",PV,412.90,400.94,64.0,88.7182,142.0000,142.0,...,0.9998,1.3018,0.7682,0.9697,0.3452,-0.3286,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00003_PV,)",KET10_1
4,KET10_1_r1c1_left,0-000-00005_PV,"(0-000-00005_PV,)",PV,342.01,390.99,80.0,88.7182,112.9000,113.0,...,0.9985,1.5163,0.6595,1.0526,0.1001,-0.6799,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00005_PV,)",KET10_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,KET9_6_r2c3_right,0-FFF-00017_WFA,"(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_...",WFA,200.33,272.90,316.0,35.7441,100.9873,95.0,...,1.0000,1.0000,1.0000,1.0128,1.2121,1.9574,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00013_PV, 0-FFF-00017_WFA)",KET9_6
10739,KET9_6_r2c3_right,0-FFF-00018_WFA,"(0-000-00003_PV, 0-FFF-00018_WFA)",WFA,208.23,372.40,400.0,35.7441,65.0975,64.0,...,0.9403,1.5236,0.6563,1.0204,0.3014,-0.221,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00003_PV, 0-FFF-00018_WFA)",KET9_6
10740,KET9_6_r2c3_right,0-FFF-00019_WFA,"(0-FFF-00019_WFA, 0-005-00068_cFos)",WFA,251.03,77.73,476.0,35.7441,55.1933,54.0,...,0.9505,1.4929,0.6698,1.0128,0.4808,0.3005,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-FFF-00019_WFA,)",KET9_6
10741,KET9_6_r2c3_right,0-FFF-00023_PV,"(0-FFF-00023_PV,)",PV,451.59,56.25,136.0,20.5509,58.6765,58.0,...,1.0000,1.1442,0.8739,1.0303,0.4208,-0.7495,KET9_6_r2c3_right_PV_sum.png,Wed Aug 07 15:15:22 PDT 2024,"(0-FFF-00023_PV,)",KET9_6


# Enforcing complete subgraph colocalization

In [10]:
df_grouped_counts = df_agg_groupings.groupby(['image_name', 'agg_grouping'])['agg_grouping']\
    .count().rename('counts').to_frame()\
    .reset_index().reset_index().drop('index', axis=1)
df_grouped_counts['len'] = df_grouped_counts.agg_grouping.apply(lambda x: len(x))

# if a grouping's length (the number of roi ids listed in the tuple) is equal to
# the number of times it appears in a given image, that grouping is plausible
# that is, if some mKate cell points to some cFos cell, and that cFos cell points
# to the same mKate cell, then that adjacency tuple must appear exactly twice in
# the given image (qualify per image here since roi_ids start from 0 for each image)

# now lets consider the case wher the counts and the lengths do not match. This 
# mismatch means that either a row was duplicated (counts > len) or that the subgraph
# defined by its adjacency tuple is not complete (counts < len); i.e. some mKate cell
# points to some cFos cell, but that cFos cell says it's single labeled. 

# duplicates where already dropped so we expect this length (counts > len) to be exactly 0
assert len(df_grouped_counts[df_grouped_counts.counts > df_grouped_counts.len]) == 0

# lets examine only cases of incomplete subgraphs
df_mismatched = df_grouped_counts[df_grouped_counts.counts < df_grouped_counts.len]
df_mismatched

,image_name,agg_grouping,counts,len
386,KET10_1_r1c2_right2,"(0-000-00007_PV, 0-001-00029_WFA)",1,2
387,KET10_1_r1c2_right2,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",1,3
422,KET10_1_r1c2_right2,"(0-001-00029_WFA, 0-005-00109_cFos)",1,2
930,KET10_1_r1c3_right,"(0-000-00010_PV, 0-001-00019_WFA)",1,2
931,KET10_1_r1c3_right,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",1,3
...,...,...,...,...
9740,KET9_6_r2c3_right,"(0-000-00009_PV, 0-001-00004_WFA, 0-005-00074_...",1,3
9741,KET9_6_r2c3_right,"(0-000-00010_PV, 0-001-00011_WFA)",1,2
9742,KET9_6_r2c3_right,"(0-000-00010_PV, 0-001-00011_WFA, 0-005-00026_...",1,3
9755,KET9_6_r2c3_right,"(0-001-00004_WFA, 0-005-00074_cFos)",1,2


### explode out all roi_ids contained in adjacency grouping tuples, merge with other data cols

In [11]:
df_coloc_mismatch = df_mismatched.explode('agg_grouping')[['image_name', 'agg_grouping']]\
    .drop_duplicates().rename(columns={'agg_grouping': 'roi_id'})\
    .merge(df_agg_groupings, how='left', on=['image_name', 'roi_id'])
    # .dropna()

df_coloc_mismatch

,image_name,roi_id,agg_grouping,stain,CoM_x,CoM_y,pixel_area,background,mean_intensity,median_intensity,...,circularity,aspect_ratio,roundness,solidity,skewness,kurtosis,filename,analysis_date,grouping,rat_n
0,KET10_1_r1c2_right2,0-000-00007_PV,"(0-000-00007_PV, 0-001-00029_WFA)",PV,328.01,120.50,97.0,52.3658,58.9691,57.0,...,0.8459,1.3997,0.6188,0.8509,0.9876,0.6663,KET10_1_r1c2_right2_PV_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-000-00007_PV,)",KET10_1
1,KET10_1_r1c2_right2,0-001-00029_WFA,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",WFA,328.58,121.33,487.0,29.9181,89.7926,84.0,...,0.9913,1.0799,0.9166,0.9939,0.6161,-0.0097,KET10_1_r1c2_right2_WFA_sum.png,Fri Aug 09 15:23:28 PDT 2024,"(0-000-00007_PV, 0-001-00029_WFA)",KET10_1
2,KET10_1_r1c2_right2,0-005-00109_cFos,"(0-001-00029_WFA, 0-005-00109_cFos)",cFos,327.45,118.18,96.0,29.9526,50.9479,49.0,...,1.0000,1.1692,0.8553,0.9796,0.5913,-0.0214,KET10_1_r1c2_right2_cfos_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-005-00109_cFos,)",KET10_1
3,KET10_1_r1c3_right,0-000-00010_PV,"(0-000-00010_PV, 0-001-00019_WFA)",PV,465.94,333.50,172.0,19.6526,88.8256,93.0,...,0.9541,1.4621,0.6840,1.0361,0.1695,-1.2699,KET10_1_r1c3_right_PV_sum.png,Wed Aug 07 09:52:11 PDT 2024,"(0-000-00010_PV,)",KET10_1
4,KET10_1_r1c3_right,0-001-00019_WFA,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",WFA,462.67,335.15,400.0,40.0169,42.6825,43.0,...,0.9403,1.5236,0.6563,1.0204,0.3468,-0.1268,KET10_1_r1c3_right_WFA_sum.png,Wed Aug 07 09:50:54 PDT 2024,"(0-000-00010_PV, 0-001-00019_WFA)",KET10_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,KET9_6_r2c3_right,0-001-00004_WFA,"(0-000-00009_PV, 0-001-00004_WFA, 0-005-00074_...",WFA,433.94,26.12,708.0,35.7441,52.6102,51.0,...,0.9994,1.1447,0.8736,1.0028,0.5384,0.6166,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00009_PV, 0-001-00004_WFA)",KET9_6
120,KET9_6_r2c3_right,0-005-00074_cFos,"(0-001-00004_WFA, 0-005-00074_cFos)",cFos,433.47,23.25,52.0,16.4019,33.9423,29.0,...,0.8124,1.3018,0.6241,0.7879,0.4868,-1.0371,KET9_6_r2c3_right_cfos_sum.png,Wed Aug 07 15:15:22 PDT 2024,"(0-005-00074_cFos,)",KET9_6
121,KET9_6_r2c3_right,0-000-00010_PV,"(0-000-00010_PV, 0-001-00011_WFA)",PV,149.17,402.36,80.0,20.5509,65.7625,63.0,...,0.9985,1.5163,0.6595,1.0526,0.4276,0.1695,KET9_6_r2c3_right_PV_sum.png,Wed Aug 07 15:15:22 PDT 2024,"(0-000-00010_PV,)",KET9_6
122,KET9_6_r2c3_right,0-001-00011_WFA,"(0-000-00010_PV, 0-001-00011_WFA, 0-005-00026_...",WFA,147.11,397.75,928.0,35.7441,57.8772,56.0,...,0.9455,1.4972,0.6679,0.9978,0.5246,0.312,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00010_PV, 0-001-00011_WFA)",KET9_6


### Get implied groupings for mismatched subgraphs

In [12]:
def implied_grouping(df, im, rid):
    '''
    makeshift breadth first search to determine implied subgraph colocalization 
    for mismatched cells
    '''
    implied_adj = [rid]
    updated_adj = []

    while set(implied_adj) != set(updated_adj):
        implied_adj += updated_adj

        for r in implied_adj:
            q = f"image_name == '{im}' and roi_id == '{r}'"
            neighbors = df.query(q)['agg_grouping']

            try: 
                neighbors = neighbors.item() 
            except: 
                print(q)
                print(neighbors,'\n')
                return('network search failed')

            for n in neighbors:
                updated_adj.append(n)

    return tuple(sorted(sorted(list(set(implied_adj))), key=lambda x: sort_order[x.split('_')[-1]]))

df_coloc_mismatch['implied_grouping'] = df_coloc_mismatch\
    .apply(lambda x: implied_grouping(df_agg_groupings, x.image_name, x.roi_id), axis=1)

## Consider differently sized groups of mismatched roi_ids separately
here we have only 2 distinct cases: the 3way case and the 4way case

In [13]:
df_coloc_mismatch['len'] = df_coloc_mismatch.implied_grouping.apply(lambda x: len(x))
print(df_coloc_mismatch.len.value_counts())

df_coloc_mismatch_3way = df_coloc_mismatch.query('len == 3')
df_coloc_mismatch_4way = df_coloc_mismatch.query('len == 4')

# check that the number of instances of each erronous implied grouping is equal 
# to the size of that grouping (i.e., an mismatched implied grouping of size 4 should
# appear exactly 4 times, once for each of the roi_id's in the grouping)
assert df_coloc_mismatch_3way.groupby('implied_grouping').implied_grouping\
    .apply(lambda x: len(x)).unique().item() == 3
assert df_coloc_mismatch_4way.groupby('implied_grouping').implied_grouping\
    .apply(lambda x: len(x)).unique().item() == 4

df_coloc_mismatch_3way.head()

len
3    120
4      4
Name: count, dtype: int64


,image_name,roi_id,agg_grouping,stain,CoM_x,CoM_y,pixel_area,background,mean_intensity,median_intensity,...,roundness,solidity,skewness,kurtosis,filename,analysis_date,grouping,rat_n,implied_grouping,len
0,KET10_1_r1c2_right2,0-000-00007_PV,"(0-000-00007_PV, 0-001-00029_WFA)",PV,328.01,120.50,97.0,52.3658,58.9691,57.0,...,0.6188,0.8509,0.9876,0.6663,KET10_1_r1c2_right2_PV_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-000-00007_PV,)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3
1,KET10_1_r1c2_right2,0-001-00029_WFA,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",WFA,328.58,121.33,487.0,29.9181,89.7926,84.0,...,0.9166,0.9939,0.6161,-0.0097,KET10_1_r1c2_right2_WFA_sum.png,Fri Aug 09 15:23:28 PDT 2024,"(0-000-00007_PV, 0-001-00029_WFA)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3
2,KET10_1_r1c2_right2,0-005-00109_cFos,"(0-001-00029_WFA, 0-005-00109_cFos)",cFos,327.45,118.18,96.0,29.9526,50.9479,49.0,...,0.8553,0.9796,0.5913,-0.0214,KET10_1_r1c2_right2_cfos_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-005-00109_cFos,)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3
3,KET10_1_r1c3_right,0-000-00010_PV,"(0-000-00010_PV, 0-001-00019_WFA)",PV,465.94,333.50,172.0,19.6526,88.8256,93.0,...,0.6840,1.0361,0.1695,-1.2699,KET10_1_r1c3_right_PV_sum.png,Wed Aug 07 09:52:11 PDT 2024,"(0-000-00010_PV,)",KET10_1,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",3
4,KET10_1_r1c3_right,0-001-00019_WFA,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",WFA,462.67,335.15,400.0,40.0169,42.6825,43.0,...,0.6563,1.0204,0.3468,-0.1268,KET10_1_r1c3_right_WFA_sum.png,Wed Aug 07 09:50:54 PDT 2024,"(0-000-00010_PV, 0-001-00019_WFA)",KET10_1,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",3


### the 3way case

In [14]:
import itertools

def dist(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def tie_breaker_3way(df_3way, current_grp):
    grp = df_3way[df_3way['implied_grouping'] == current_grp].copy(deep=True)
    coords = dict(zip(grp.roi_id,list(zip(grp.CoM_x.values, grp.CoM_y.values))))

    distances = []
    for p1, p2 in itertools.combinations(coords.keys(), 2):
        stain_type1 = p1.split('_')[-1]
        stain_type2 = p2.split('_')[-1]
        if not stain_type1 == stain_type2:
            distances.append(((p1, p2), dist(coords[p1], coords[p2])))
        else:
            print(f'{p1} and {p2} cannot be colocalized; skipping distance computation for this pair')

    d = dict(distances)
    winner = set(min(d, key=d.get))
    leftover = set(current_grp) - winner

    winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
    leftover = tuple(leftover)

    # update groupings
    grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else leftover, axis=1)

    return grp

mismatched_3ways = df_coloc_mismatch_3way.implied_grouping.unique()
tie_broken_3ways = [tie_breaker_3way(df_coloc_mismatch_3way, grp) for grp in mismatched_3ways]
df_3way_tiebreak = pd.concat(tie_broken_3ways)

0-001-00017_WFA and 0-001-00027_WFA cannot be colocalized; skipping distance computation for this pair


### the 4way case

In [15]:
def tie_breaker_4way(df_4way, current_grp):
    def check_same(leftover):
        p1, p2 = leftover
        stain_type1 = p1.split('_')[-1]
        stain_type2 = p2.split('_')[-1]

        if not stain_type1 == stain_type2:
            return False
        else:
            return True

    # after computing winning pair, we have 3 possible out comes:
    # first leftover rids form a true pair
    def check_pair(leftover):
        p1, p2 = leftover
        if p2 in grp[grp.roi_id == p1]['grouping'].item() and p1 in grp[grp.roi_id == p2]['grouping'].item():
            return True
        else:
            return False

    def check_triple(leftover, winner):
        l1, l2 = leftover
        w1, w2 = winner

        if w1 in grp[grp.roi_id == l1]['grouping'].item() \
            and w2 in grp[grp.roi_id == l1]['grouping'].item() \
            and l1 in grp[grp.roi_id == w1]['grouping'].item() \
            and l1 in grp[grp.roi_id == w2]['grouping'].item():
            return True, l1

        elif w1 in grp[grp.roi_id == l2]['grouping'].item() \
            and w2 in grp[grp.roi_id == l2]['grouping'].item() \
            and l2 in grp[grp.roi_id == w1]['grouping'].item() \
            and l2 in grp[grp.roi_id == w2]['grouping'].item():
            return True, l2

        else:
            return False, None

    grp = df_4way[df_4way['implied_grouping'] == current_grp].copy(deep=True)
    coords = dict(zip(grp.roi_id,list(zip(grp.CoM_x.values, grp.CoM_y.values))))

    distances = []
    for p1, p2 in itertools.combinations(coords.keys(), 2):
        stain_type1 = p1.split('_')[-1]
        stain_type2 = p2.split('_')[-1]
        if not stain_type1 == stain_type2:
            distances.append(((p1, p2), dist(coords[p1], coords[p2])))
        else:
            print(f'{p1} and {p2} cannot be colocalized; skipping distance computation for this pair')

    d = dict(distances)
    winning_set = set(min(d, key=d.get))
    winner = tuple(winning_set)
    leftover = tuple(set(current_grp) - winning_set)
    lonely_leftovers = False

    # if leftover rids not the same staintype, check if they belong in a separate pair
    if not check_same(leftover):
        if not check_pair(leftover):

            # if leftover rids not the same staintype, and they do NOT form a pair,
            # check of the winning pair should be triple
            triple, triple_rid = check_triple(leftover, winner)

            # if winning pair should be a triple, add that rid to winning tuple
            if triple:
                winner += (triple_rid,)
                leftover = tuple(set(leftover) - set(winner))
            
            # if leftover rids do not form a pair and winner is not triple, then split them
            elif not triple:
                lonely_leftovers = True

        # leftover rids not the same staintype, and both belong in a separate pair
        elif check_pair(leftover):
            # keep pairs as is
            winner = winner
            leftover = leftover

    # if both leftover rids are the same staintype, the winning pair could still be a triple
    elif check_same(leftover):
        triple, triple_rid = check_triple(leftover, winner)

        # if winning pair should be a triple, add that rid to winning tuple
        if triple:
            winner += (triple_rid,)
            leftover = tuple(set(leftover) - set(winner))
        
        # if the winning pair is NOT triple and the leftover rids are the SAME staintype, split them
        elif not triple:
            lonely_leftovers = True

    if not lonely_leftovers:
        winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
        leftover = tuple(sorted([rid for rid in leftover], key=lambda x: sort_order[x.split('_')[-1]]))
        grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else leftover, axis=1)

    elif lonely_leftovers:
        winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
        grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else (x.roi_id,), axis=1)

    return grp

mismatched_4ways = df_coloc_mismatch_4way.implied_grouping.unique()
tie_broken_4ways = [tie_breaker_4way(df_coloc_mismatch_4way, grp) for grp in mismatched_4ways]
df_4way_tiebreak = pd.concat(tie_broken_4ways)

0-005-00023_cFos and 0-005-00178_cFos cannot be colocalized; skipping distance computation for this pair


### the 5way case
luckily, we do not have the 5way case in this data set, but for completeness the code for this case is included below

In [16]:
def tie_breaker_5way(df_5way, current_grp):
    def check_same(leftover):
        p1, p2 = leftover
        stain_type1 = p1.split('_')[-1]
        stain_type2 = p2.split('_')[-1]

        if not stain_type1 == stain_type2:
            return False
        else:
            return True

    # after computing winning pair, we have 3 possible out comes:
    # first leftover rids form a true pair
    def check_pair(leftover):
        p1, p2 = leftover
        if p2 in grp[grp.roi_id == p1]['grouping'].item() and p1 in grp[grp.roi_id == p2]['grouping'].item():
            return True
        else:
            return False

    def check_triple(leftover, winner):
        if len(leftover) == 3:
            l1, l2, l3 = leftover
            w1, w2 = winner

            if w1 in grp[grp.roi_id == l1]['grouping'].item() \
                and w2 in grp[grp.roi_id == l1]['grouping'].item() \
                and l1 in grp[grp.roi_id == w1]['grouping'].item() \
                and l1 in grp[grp.roi_id == w2]['grouping'].item():
                return True, l1

            elif w1 in grp[grp.roi_id == l2]['grouping'].item() \
                and w2 in grp[grp.roi_id == l2]['grouping'].item() \
                and l2 in grp[grp.roi_id == w1]['grouping'].item() \
                and l2 in grp[grp.roi_id == w2]['grouping'].item():
                return True, l2
            
            elif w1 in grp[grp.roi_id == l3]['grouping'].item() \
                and w2 in grp[grp.roi_id == l3]['grouping'].item() \
                and l3 in grp[grp.roi_id == w1]['grouping'].item() \
                and l3 in grp[grp.roi_id == w2]['grouping'].item():
                return True, l3
            
            else:
                return False, None
        
        elif len(leftover) == 1:
            l1 = leftover[0]
            w1, w2 = winner

            if w1 in grp[grp.roi_id == l1]['grouping'].item() \
                and w2 in grp[grp.roi_id == l1]['grouping'].item() \
                and l1 in grp[grp.roi_id == w1]['grouping'].item() \
                and l1 in grp[grp.roi_id == w2]['grouping'].item():
                return True, l1

            else:
                return False, None

    def check_lonely(leftover):

        p1, p2, p3 = leftover
        # leftover rids are lonely if none of them point to each other
        if p1 in grp[grp.roi_id == p2]['grouping'].item() \
            or p1 in grp[grp.roi_id == p3]['grouping'].item() \
            or p2 in grp[grp.roi_id == p1]['grouping'].item() \
            or p2 in grp[grp.roi_id == p3]['grouping'].item() \
            or p3 in grp[grp.roi_id == p1]['grouping'].item() \
            or p3 in grp[grp.roi_id == p2]['grouping'].item():
            return False
        else:
            return True

    def map_updated_groupings(roi_id, groups):
        for g in groups:
            if roi_id in g:
                return g

    grp = df_5way[df_5way['implied_grouping'] == current_grp].copy(deep=True)
    coords = dict(zip(grp.roi_id,list(zip(grp.CoM_x.values, grp.CoM_y.values))))

    distances = []
    for p1, p2 in itertools.combinations(coords.keys(), 2):
        stain_type1 = p1.split('_')[-1]
        stain_type2 = p2.split('_')[-1]
        if not stain_type1 == stain_type2:
            distances.append(((p1, p2), dist(coords[p1], coords[p2])))
        else:
            print(f'{p1} and {p2} cannot be colocalized; skipping distance computation for this pair')

    d = dict(distances)
    winning_set = set(min(d, key=d.get))

    winner = tuple(winning_set)
    leftover = tuple(set(current_grp) - winning_set)
    lonely_leftovers = False
    next_winner = False

    # check if winning pair is triple
    triple, triple_rid = check_triple(leftover, winner)
    if triple:
        # add triple_rid to winner, remove new winner
        winner += (triple_rid,)
        leftover = tuple(set(leftover) - set(winner))

        # check if remaining two leftover rids form a pair
        if check_pair(leftover):
            # do nothing and update groupings
            winner = winner
            leftover = leftover

        elif not check_pair(leftover):
            lonely_leftovers = True

    elif not triple:
        # check if we have the lonely case
        if not check_lonely(leftover):
            # if first winner is not triple and we do not have the lonely case,
            # there must be at least one other true pairing; get the next closest pair

            # continue to get next closest pair until winner and next_winner do not overlap
            next_winner = winner
            while len(set(winner).intersection(set(next_winner))) > 0:
                del d[min(d, key=d.get)]
                next_winning_set = set(min(d, key=d.get))
                next_winner = tuple(next_winning_set)
            leftover = tuple(set(leftover) - next_winning_set)

            # check if next winner is triple
            next_triple, next_triple_rid =  check_triple(leftover, next_winner)
            if next_triple:
                # add next_triple_rid to next_winner, remove new winner
                next_winner += (next_triple_rid,)
                leftover = tuple(set(leftover) - set(next_winner))

            # if it is not, then we have 2 pairs and one single lonely cell
            elif not next_triple:
                # do nothing and updated groupings
                next_winner = next_winner
                leftover = leftover

        else:
            lonely_leftovers = True

    ### update groupings
    # the case where our first winner was triple, and the leftovers formed a pair
    if not next_winner and not lonely_leftovers:
        winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
        leftover = tuple(sorted([rid for rid in leftover], key=lambda x: sort_order[x.split('_')[-1]]))
        grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else leftover, axis=1)

    # the case where our first winner was a pair, and the leftovers either formed a pair or triple
    if next_winner:
        winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
        next_winner = tuple(sorted([rid for rid in next_winner], key=lambda x: sort_order[x.split('_')[-1]]))

        # the case where our next winner was a pair, and we had one single leftover
        if len(leftover) > 0:
            leftover = tuple(sorted([rid for rid in leftover], key=lambda x: sort_order[x.split('_')[-1]]))
            grp['updated_grouping'] = grp.apply(lambda x: map_updated_groupings(x.roi_id, [winner, next_winner, leftover]), axis=1)
        
        # the case where our next winner was a triple and we had no singles leftover
        else:
            grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else next_winner, axis=1)

    # the case where our first winner was a pair and all remaining leftovers were lonely
    elif lonely_leftovers:
        winner = tuple(sorted([rid for rid in winner], key=lambda x: sort_order[x.split('_')[-1]]))
        grp['updated_grouping'] = grp.apply(lambda x: winner if x.roi_id in winner else (x.roi_id,), axis=1)
    
    return grp

In [17]:
# final concatentation
df_tiebreak = pd.concat([df_3way_tiebreak,df_4way_tiebreak])
df_tiebreak

,image_name,roi_id,agg_grouping,stain,CoM_x,CoM_y,pixel_area,background,mean_intensity,median_intensity,...,solidity,skewness,kurtosis,filename,analysis_date,grouping,rat_n,implied_grouping,len,updated_grouping
0,KET10_1_r1c2_right2,0-000-00007_PV,"(0-000-00007_PV, 0-001-00029_WFA)",PV,328.01,120.50,97.0,52.3658,58.9691,57.0,...,0.8509,0.9876,0.6663,KET10_1_r1c2_right2_PV_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-000-00007_PV,)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3,"(0-000-00007_PV, 0-001-00029_WFA)"
1,KET10_1_r1c2_right2,0-001-00029_WFA,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",WFA,328.58,121.33,487.0,29.9181,89.7926,84.0,...,0.9939,0.6161,-0.0097,KET10_1_r1c2_right2_WFA_sum.png,Fri Aug 09 15:23:28 PDT 2024,"(0-000-00007_PV, 0-001-00029_WFA)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3,"(0-000-00007_PV, 0-001-00029_WFA)"
2,KET10_1_r1c2_right2,0-005-00109_cFos,"(0-001-00029_WFA, 0-005-00109_cFos)",cFos,327.45,118.18,96.0,29.9526,50.9479,49.0,...,0.9796,0.5913,-0.0214,KET10_1_r1c2_right2_cfos_sum.png,Fri Aug 09 15:23:54 PDT 2024,"(0-005-00109_cFos,)",KET10_1,"(0-000-00007_PV, 0-001-00029_WFA, 0-005-00109_...",3,"(0-005-00109_cFos,)"
3,KET10_1_r1c3_right,0-000-00010_PV,"(0-000-00010_PV, 0-001-00019_WFA)",PV,465.94,333.50,172.0,19.6526,88.8256,93.0,...,1.0361,0.1695,-1.2699,KET10_1_r1c3_right_PV_sum.png,Wed Aug 07 09:52:11 PDT 2024,"(0-000-00010_PV,)",KET10_1,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",3,"(0-000-00010_PV, 0-001-00019_WFA)"
4,KET10_1_r1c3_right,0-001-00019_WFA,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",WFA,462.67,335.15,400.0,40.0169,42.6825,43.0,...,1.0204,0.3468,-0.1268,KET10_1_r1c3_right_WFA_sum.png,Wed Aug 07 09:50:54 PDT 2024,"(0-000-00010_PV, 0-001-00019_WFA)",KET10_1,"(0-000-00010_PV, 0-001-00019_WFA, 0-005-00047_...",3,"(0-000-00010_PV, 0-001-00019_WFA)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,KET9_6_r2c3_right,0-005-00026_cFos,"(0-001-00011_WFA, 0-005-00026_cFos)",cFos,132.90,389.50,39.0,16.4019,30.2564,29.0,...,0.5909,0.1574,-1.2615,KET9_6_r2c3_right_cfos_sum.png,Wed Aug 07 15:15:22 PDT 2024,"(0-005-00026_cFos,)",KET9_6,"(0-000-00010_PV, 0-001-00011_WFA, 0-005-00026_...",3,"(0-005-00026_cFos,)"
9,KET11_10_r1c3_right,0-000-00002_PV,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",PV,290.56,308.01,84.0,35.3117,98.6429,95.0,...,0.9333,0.6624,-0.3382,KET11_10_r1c3_right_PV_sum.png,Wed Aug 07 13:04:56 PDT 2024,"(0-000-00002_PV, 0-005-00178_cFos)",KET11_10,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",4,"(0-000-00002_PV, 0-005-00178_cFos)"
10,KET11_10_r1c3_right,0-001-00006_WFA,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",WFA,294.26,307.04,792.0,54.4198,81.5581,76.0,...,0.9802,0.8824,0.7436,KET11_10_r1c3_right_WFA_sum.png,Wed Aug 07 10:46:50 PDT 2024,"(0-000-00002_PV, 0-001-00006_WFA)",KET11_10,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",4,"(0-001-00006_WFA,)"
11,KET11_10_r1c3_right,0-005-00023_cFos,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",cFos,293.73,317.14,65.0,18.0020,24.6000,24.0,...,0.8553,0.6184,0.0382,KET11_10_r1c3_right_cfos_sum.png,Wed Aug 07 13:04:56 PDT 2024,"(0-005-00023_cFos,)",KET11_10,"(0-000-00002_PV, 0-001-00006_WFA, 0-005-00023_...",4,"(0-005-00023_cFos,)"


### update groupings with our new true groupings

In [18]:
df_tiebreak['iid_rid'] = df_tiebreak[['image_name', 'roi_id']].agg('_'.join, axis=1)
df_agg_groupings['iid_rid'] = df_agg_groupings[['image_name', 'roi_id']].agg('_'.join, axis=1)

df_true_grp = df_agg_groupings.merge(df_tiebreak[['iid_rid', 'updated_grouping']].copy(), how='left', on='iid_rid')
df_true_grp['true_grouping'] = df_true_grp.updated_grouping.fillna(df_true_grp.agg_grouping)
df_true_grp

,image_name,roi_id,agg_grouping,stain,CoM_x,CoM_y,pixel_area,background,mean_intensity,median_intensity,...,solidity,skewness,kurtosis,filename,analysis_date,grouping,rat_n,iid_rid,updated_grouping,true_grouping
0,KET10_1_r1c1_left,0-000-00000_PV,"(0-000-00000_PV,)",PV,502.88,236.10,80.0,88.7182,127.6000,126.0,...,0.9524,0.6552,1.0906,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00000_PV,)",KET10_1,KET10_1_r1c1_left_0-000-00000_PV,NaN,"(0-000-00000_PV,)"
1,KET10_1_r1c1_left,0-000-00001_PV,"(0-000-00001_PV,)",PV,491.89,306.14,80.0,88.7182,114.3500,115.0,...,1.0526,0.0226,0.1296,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00001_PV,)",KET10_1,KET10_1_r1c1_left_0-000-00001_PV,NaN,"(0-000-00001_PV,)"
2,KET10_1_r1c1_left,0-000-00002_PV,"(0-000-00002_PV,)",PV,436.16,167.66,80.0,88.7182,129.1875,136.0,...,0.9524,-0.4449,-0.7357,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00002_PV,)",KET10_1,KET10_1_r1c1_left_0-000-00002_PV,NaN,"(0-000-00002_PV,)"
3,KET10_1_r1c1_left,0-000-00003_PV,"(0-000-00003_PV,)",PV,412.90,400.94,64.0,88.7182,142.0000,142.0,...,0.9697,0.3452,-0.3286,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00003_PV,)",KET10_1,KET10_1_r1c1_left_0-000-00003_PV,NaN,"(0-000-00003_PV,)"
4,KET10_1_r1c1_left,0-000-00005_PV,"(0-000-00005_PV,)",PV,342.01,390.99,80.0,88.7182,112.9000,113.0,...,1.0526,0.1001,-0.6799,KET10_1_r1c1_left_PV_sum.png,Wed Aug 07 09:14:48 PDT 2024,"(0-000-00005_PV,)",KET10_1,KET10_1_r1c1_left_0-000-00005_PV,NaN,"(0-000-00005_PV,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,KET9_6_r2c3_right,0-FFF-00017_WFA,"(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_...",WFA,200.33,272.90,316.0,35.7441,100.9873,95.0,...,1.0128,1.2121,1.9574,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00013_PV, 0-FFF-00017_WFA)",KET9_6,KET9_6_r2c3_right_0-FFF-00017_WFA,NaN,"(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_..."
10739,KET9_6_r2c3_right,0-FFF-00018_WFA,"(0-000-00003_PV, 0-FFF-00018_WFA)",WFA,208.23,372.40,400.0,35.7441,65.0975,64.0,...,1.0204,0.3014,-0.221,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-000-00003_PV, 0-FFF-00018_WFA)",KET9_6,KET9_6_r2c3_right_0-FFF-00018_WFA,NaN,"(0-000-00003_PV, 0-FFF-00018_WFA)"
10740,KET9_6_r2c3_right,0-FFF-00019_WFA,"(0-FFF-00019_WFA, 0-005-00068_cFos)",WFA,251.03,77.73,476.0,35.7441,55.1933,54.0,...,1.0128,0.4808,0.3005,KET9_6_r2c3_right_WFA_sum.png,Wed Aug 07 15:15:04 PDT 2024,"(0-FFF-00019_WFA,)",KET9_6,KET9_6_r2c3_right_0-FFF-00019_WFA,NaN,"(0-FFF-00019_WFA, 0-005-00068_cFos)"
10741,KET9_6_r2c3_right,0-FFF-00023_PV,"(0-FFF-00023_PV,)",PV,451.59,56.25,136.0,20.5509,58.6765,58.0,...,1.0303,0.4208,-0.7495,KET9_6_r2c3_right_PV_sum.png,Wed Aug 07 15:15:22 PDT 2024,"(0-FFF-00023_PV,)",KET9_6,KET9_6_r2c3_right_0-FFF-00023_PV,NaN,"(0-FFF-00023_PV,)"


# Build dummy coloc cols
exploding out the colocalization groupings this way makes filtering much easier

In [19]:
def get_dummies(x):
    groupings = [rid.split('_')[-1] for rid in x]

    dummy_PV = False
    dummy_WFA = False
    dummy_cFos = False

    if 'PV' in groupings:
        dummy_PV = True
    if 'WFA' in groupings:
        dummy_WFA = True
    if 'cFos' in groupings:
        dummy_cFos = True

    return dummy_PV, dummy_WFA, dummy_cFos

df_true_grp['dummy'] = df_true_grp.true_grouping.apply(get_dummies)
df_true_grp['dummy_PV'], df_true_grp['dummy_WFA'], df_true_grp['dummy_cFos'] = zip(*df_true_grp['dummy'])

# reorder cols
df_true_grp = df_true_grp['iid_rid dummy_PV dummy_WFA dummy_cFos image_name roi_id stain CoM_x CoM_y background mean_intensity mean-background filename rat_n grouping agg_grouping updated_grouping true_grouping'.split()]
df_true_grp

,iid_rid,dummy_PV,dummy_WFA,dummy_cFos,image_name,roi_id,stain,CoM_x,CoM_y,background,mean_intensity,mean-background,filename,rat_n,grouping,agg_grouping,updated_grouping,true_grouping
0,KET10_1_r1c1_left_0-000-00000_PV,True,False,False,KET10_1_r1c1_left,0-000-00000_PV,PV,502.88,236.10,88.7182,127.6000,35.8621,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00000_PV,)","(0-000-00000_PV,)",NaN,"(0-000-00000_PV,)"
1,KET10_1_r1c1_left_0-000-00001_PV,True,False,False,KET10_1_r1c1_left,0-000-00001_PV,PV,491.89,306.14,88.7182,114.3500,28.1585,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00001_PV,)","(0-000-00001_PV,)",NaN,"(0-000-00001_PV,)"
2,KET10_1_r1c1_left_0-000-00002_PV,True,False,False,KET10_1_r1c1_left,0-000-00002_PV,PV,436.16,167.66,88.7182,129.1875,37.5041,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00002_PV,)","(0-000-00002_PV,)",NaN,"(0-000-00002_PV,)"
3,KET10_1_r1c1_left_0-000-00003_PV,True,False,False,KET10_1_r1c1_left,0-000-00003_PV,PV,412.90,400.94,88.7182,142.0000,50.6469,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00003_PV,)","(0-000-00003_PV,)",NaN,"(0-000-00003_PV,)"
4,KET10_1_r1c1_left_0-000-00005_PV,True,False,False,KET10_1_r1c1_left,0-000-00005_PV,PV,342.01,390.99,88.7182,112.9000,22.8983,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00005_PV,)","(0-000-00005_PV,)",NaN,"(0-000-00005_PV,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,KET9_6_r2c3_right_0-FFF-00017_WFA,True,True,True,KET9_6_r2c3_right,0-FFF-00017_WFA,WFA,200.33,272.90,35.7441,100.9873,66.2054,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-000-00013_PV, 0-FFF-00017_WFA)","(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_...",NaN,"(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_..."
10739,KET9_6_r2c3_right_0-FFF-00018_WFA,True,True,False,KET9_6_r2c3_right,0-FFF-00018_WFA,WFA,208.23,372.40,35.7441,65.0975,29.9464,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-000-00003_PV, 0-FFF-00018_WFA)","(0-000-00003_PV, 0-FFF-00018_WFA)",NaN,"(0-000-00003_PV, 0-FFF-00018_WFA)"
10740,KET9_6_r2c3_right_0-FFF-00019_WFA,False,True,True,KET9_6_r2c3_right,0-FFF-00019_WFA,WFA,251.03,77.73,35.7441,55.1933,19.1788,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-FFF-00019_WFA,)","(0-FFF-00019_WFA, 0-005-00068_cFos)",NaN,"(0-FFF-00019_WFA, 0-005-00068_cFos)"
10741,KET9_6_r2c3_right_0-FFF-00023_PV,True,False,False,KET9_6_r2c3_right,0-FFF-00023_PV,PV,451.59,56.25,20.5509,58.6765,37.6781,KET9_6_r2c3_right_PV_sum.png,KET9_6,"(0-FFF-00023_PV,)","(0-FFF-00023_PV,)",NaN,"(0-FFF-00023_PV,)"


#### spot checking

In [20]:
# do our doubles agree?
def check_double_ns(df_true):
    print('double labeled ns: ')
    for stain_x, stain_y in itertools.combinations(['PV', 'WFA', 'cFos'], r=2):
        x_on_y = df_true.query(f'dummy_{stain_x} == True and dummy_{stain_y} == True and stain == "{stain_x}"')
        y_on_x = df_true.query(f'dummy_{stain_x} == True and dummy_{stain_y} == True and stain == "{stain_y}"')
        # diff = x_on_y.__len__() - y_on_x.__len__()
        print(f'{stain_x} on {stain_y}:    {x_on_y.__len__()}')
        print(f'{stain_y} on {stain_x}:    {y_on_x.__len__()}\n')

# do our triples agree?
def check_triple_ns(comb, df_true):
    stain_x, stain_y, stain_z = comb
    q = df_true.query(
        f'dummy_{stain_x} == True and dummy_{stain_y} == True and dummy_{stain_z} == True and\
         (stain == "{stain_x}" or stain == "{stain_y}" or stain == "{stain_z}")'
    )

    q_x = q.query(f'stain == "{stain_x}"')
    q_y = q.query(f'stain == "{stain_y}"')
    q_z = q.query(f'stain == "{stain_z}"')

    print(f'\ntriple {stain_x},{stain_y},{stain_z} ns:')
    print(stain_x, ' :', q_x.__len__())
    print(stain_y, ' :', q_y.__len__())
    print(stain_z, ' :', q_z.__len__())

check_double_ns(df_true_grp)

for comb in itertools.combinations(['PV', 'WFA', 'cFos'], r=3):
    check_triple_ns(comb, df_true_grp)

double labeled ns: 
PV on WFA:    477
WFA on PV:    477

PV on cFos:    292
cFos on PV:    292

WFA on cFos:    345
cFos on WFA:    345


triple PV,WFA,cFos ns:
PV  : 150
WFA  : 150
cFos  : 150


### We have the cohort key !
All of these rats should have received a VR5 reactivation session with some of them having recieved either an IP injection of either saline or ketamine. Some rats from AG's previous gruops were also added as controls.

In [21]:
# prior to reading the csv here, I added new rows in this csv for the new KET-11 group
df_key = pd.read_csv('KET PRE VR5 COHORT KEY.csv')
display(df_key.head())

# matching string formatting for use as keys
df_key.Subject = df_key.Subject.apply(lambda x: x.replace('-','_').replace('KET_','KET').replace('PE_','PE'))

# set up df_key for merge
df_key['rat_n'] = df_key.Subject
df_key['react'] = df_key.TX.apply(lambda x: x.split(' ')[0])
df_key['treat'] = df_key.TX.apply(lambda x: x.split(' ')[-1])
df_key = df_key.drop(['Subject', 'TX', 'Cue'], axis=1)
display(df_key.head())

# add new react and treat cols to our large dataframe
df_true_grp = df_true_grp.merge(df_key, how='left', on='rat_n')

### GREAT. now we can write this to disk and proceed with our analyses
df_true_grp.to_csv('KET_VR5_SAC_FINAL.csv')
df_true_grp

,Subject,TX,Cue
0,PE-11-1,FR1 KET,9.0
1,PE-11-2,FR1 KET,25.0
2,PE-11-3,FR1 KET,21.0
3,PE-12-1,FR1 KET,45.0
4,PE-12-2,FR1 KET,31.0


,rat_n,react,treat
0,PE11_1,FR1,KET
1,PE11_2,FR1,KET
2,PE11_3,FR1,KET
3,PE12_1,FR1,KET
4,PE12_2,FR1,KET


,iid_rid,dummy_PV,dummy_WFA,dummy_cFos,image_name,roi_id,stain,CoM_x,CoM_y,background,mean_intensity,mean-background,filename,rat_n,grouping,agg_grouping,updated_grouping,true_grouping,react,treat
0,KET10_1_r1c1_left_0-000-00000_PV,True,False,False,KET10_1_r1c1_left,0-000-00000_PV,PV,502.88,236.10,88.7182,127.6000,35.8621,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00000_PV,)","(0-000-00000_PV,)",NaN,"(0-000-00000_PV,)",FR1,SAL
1,KET10_1_r1c1_left_0-000-00001_PV,True,False,False,KET10_1_r1c1_left,0-000-00001_PV,PV,491.89,306.14,88.7182,114.3500,28.1585,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00001_PV,)","(0-000-00001_PV,)",NaN,"(0-000-00001_PV,)",FR1,SAL
2,KET10_1_r1c1_left_0-000-00002_PV,True,False,False,KET10_1_r1c1_left,0-000-00002_PV,PV,436.16,167.66,88.7182,129.1875,37.5041,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00002_PV,)","(0-000-00002_PV,)",NaN,"(0-000-00002_PV,)",FR1,SAL
3,KET10_1_r1c1_left_0-000-00003_PV,True,False,False,KET10_1_r1c1_left,0-000-00003_PV,PV,412.90,400.94,88.7182,142.0000,50.6469,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00003_PV,)","(0-000-00003_PV,)",NaN,"(0-000-00003_PV,)",FR1,SAL
4,KET10_1_r1c1_left_0-000-00005_PV,True,False,False,KET10_1_r1c1_left,0-000-00005_PV,PV,342.01,390.99,88.7182,112.9000,22.8983,KET10_1_r1c1_left_PV_sum.png,KET10_1,"(0-000-00005_PV,)","(0-000-00005_PV,)",NaN,"(0-000-00005_PV,)",FR1,SAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10738,KET9_6_r2c3_right_0-FFF-00017_WFA,True,True,True,KET9_6_r2c3_right,0-FFF-00017_WFA,WFA,200.33,272.90,35.7441,100.9873,66.2054,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-000-00013_PV, 0-FFF-00017_WFA)","(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_...",NaN,"(0-000-00013_PV, 0-FFF-00017_WFA, 0-005-00040_...",FR1,SAL
10739,KET9_6_r2c3_right_0-FFF-00018_WFA,True,True,False,KET9_6_r2c3_right,0-FFF-00018_WFA,WFA,208.23,372.40,35.7441,65.0975,29.9464,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-000-00003_PV, 0-FFF-00018_WFA)","(0-000-00003_PV, 0-FFF-00018_WFA)",NaN,"(0-000-00003_PV, 0-FFF-00018_WFA)",FR1,SAL
10740,KET9_6_r2c3_right_0-FFF-00019_WFA,False,True,True,KET9_6_r2c3_right,0-FFF-00019_WFA,WFA,251.03,77.73,35.7441,55.1933,19.1788,KET9_6_r2c3_right_WFA_sum.png,KET9_6,"(0-FFF-00019_WFA,)","(0-FFF-00019_WFA, 0-005-00068_cFos)",NaN,"(0-FFF-00019_WFA, 0-005-00068_cFos)",FR1,SAL
10741,KET9_6_r2c3_right_0-FFF-00023_PV,True,False,False,KET9_6_r2c3_right,0-FFF-00023_PV,PV,451.59,56.25,20.5509,58.6765,37.6781,KET9_6_r2c3_right_PV_sum.png,KET9_6,"(0-FFF-00023_PV,)","(0-FFF-00023_PV,)",NaN,"(0-FFF-00023_PV,)",FR1,SAL
